In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
app = pd.read_csv('googleplaystore.csv')
app.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [4]:
review=pd.read_csv('googleplaystore_user_reviews.csv')
review=review.drop_duplicates() # removed any duplicate ratings
review=review.dropna(subset=['Sentiment'], inplace=False) # removed any empty ratings
review.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000
5,10 Best Foods for You,Best way,Positive,1.00,0.300000


In [6]:
review=review.drop(['Translated_Review','Sentiment'],axis='columns') # To get only app, polarity and, subjectivity

In [7]:
review['Number of written']=review.groupby('App').count()

,Sentiment_Polarity,Sentiment_Subjectivity
App,,
10 Best Foods for You,95,95
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,36,36
11st,40,40
1800 Contacts - Lens Store,40,40
1LINE – One Line with One Touch,38,38
...,...,...
Hotels.com: Book Hotel Rooms & Find Vacation Deals,39,39
Hotspot Shield Free VPN Proxy & Wi-Fi Security,32,32
Hotstar,32,32


In [176]:
pol=review.groupby('App')['Sentiment_Polarity'].mean().reset_index() # grouped for polarity
sub=review.groupby('App')['Sentiment_Subjectivity'].mean().reset_index() # grouped for subjectivity
review=pd.merge(pol,sub,on='App') #merged into a dataset
review

,App,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,0.465906,0.493254
1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,0.358227,0.508907
2,11st,0.181294,0.443957
3,1800 Contacts - Lens Store,0.318145,0.591098
4,1LINE – One Line with One Touch,0.196290,0.557315
...,...,...,...
860,Hotels.com: Book Hotel Rooms & Find Vacation D...,0.098559,0.525508
861,Hotspot Shield Free VPN Proxy & Wi-Fi Security,0.223750,0.380365
862,Hotstar,0.038178,0.493964
863,Hotwire Hotel & Car Rental App,0.187029,0.459717


In [177]:
#polarity is how positive the review is, 
#the higher the subjectivity, the more subjective the review is so we reverse by doing 1-subjectivity
#this is because more objectivity holds more weight
review['Overall']=review['Sentiment_Polarity']*(1-review['Sentiment_Subjectivity']) #polarity*(1-subjectivity)
review=review.drop(['Sentiment_Polarity','Sentiment_Subjectivity'],axis=1)

In [178]:
app_rev=pd.merge(app,review,on='App') # we will now only have to work with one dataset
app_rev[['Rating','Overall']].interpolate()# interpolate any missing ratings based on the written reviews

,Rating,Overall
0,3.9,0.041779
1,4.4,0.127946
2,4.3,0.132367
3,4.1,0.137321
4,4.7,0.073415
...,...,...
1224,4.0,-0.034154
1225,4.1,0.138462
1226,4.5,0.037900
1227,4.1,0.092807


In [183]:
app_rev.isna().sum() # rating has 1 (don't know why as there are no empty)

App               0
Category          0
Rating            1
Reviews           0
Size              0
Installs          0
Type              0
Price             0
Content Rating    0
Genres            0
Last Updated      0
Current Ver       0
Android Ver       0
Overall           0
dtype: int64

In [184]:
app_rev.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Overall
0,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,0.041779
1,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up,0.127946
2,FlipaClip - Cartoon animation,ART_AND_DESIGN,4.3,194216,39M,"5,000,000+",Free,0,Everyone,Art & Design,"August 3, 2018",2.2.5,4.0.3 and up,0.132367
3,Boys Photo Editor - Six Pack & Men's Suit,ART_AND_DESIGN,4.1,654,12M,"100,000+",Free,0,Everyone,Art & Design,"March 20, 2018",1.1,4.0.3 and up,0.137321
4,Colorfit - Drawing & Coloring,ART_AND_DESIGN,4.7,20260,25M,"500,000+",Free,0,Everyone,Art & Design;Creativity,"October 11, 2017",1.0.8,4.0.3 and up,0.073415
